In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
%env SOCCERDATA_LOGLEVEL=ERROR
%env SOCCERDATA_NOCACHE=True
%env SOCCERDATA_NOSTORE=True

env: SOCCERDATA_LOGLEVEL=ERROR
env: SOCCERDATA_NOCACHE=True
env: SOCCERDATA_NOSTORE=True


In [3]:
import soccerdata as sd

In [5]:
five38 = sd.FiveThirtyEight(leagues="ES-La Liga", seasons=2021)
print(five38.__doc__)

ValueError: 
                        Invalid league 'ES-La Liga'. Valid leagues are:
                        ['ENG-Premier League',
 'ESP-La Liga',
 'FRA-Ligue 1',
 'GER-Bundesliga',
 'ITA-Serie A']
                        